In [1]:
import sys
import argparse
import os
from yolo import YOLO, detect_video
from PIL import Image
import numpy as np
import json

Using TensorFlow backend.
/home/hscc_yuchen/anaconda3/envs/dvl_hw2_svhn/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/hscc_yuchen/anaconda3/envs/dvl_hw2_svhn/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/hscc_yuchen/anaconda3/envs/dvl_hw2_svhn/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np

/home/hscc_yuchen/anaconda3/envs/dvl_hw2_svhn/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/hscc_yuchen/anaconda3/envs/dvl_hw2_svhn/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/hscc_yuchen/anaconda3/envs/dvl_hw2_svhn/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int1

In [2]:
def detect_img(yolo):
    while True:
        img = input('Input image filename:')
        try:
            image = Image.open(img)
        except:
            print('Open Error! Try again!')
            continue
        else:
            r_image = yolo.detect_image(image)
            r_image.show()
    yolo.close_session()

In [3]:
def get_image_files(dir):
    imgs = []
    file_dir = os.listdir(dir)
    file_dir.sort(key=lambda x:int(x[:-4]))
    #print(file_dir)
    for file in file_dir:
        #print(file[:-4])
        file_lower = file.lower()
        if file_lower.endswith(".png") or file_lower.endswith(".jpg"):
            imgs.append(os.path.join(dir, file))
    return imgs

In [4]:
def detect_imgdir(yolo, dir, output_txt=False):
    sub_dict = {"bbox":[], "label":[], "score":[]}
    submission = []
    speed_set = []
    img_files = get_image_files(dir)
    save_dir = os.path.join('output')
    if not os.path.exists(save_dir): os.makedirs(save_dir)
    for img in img_files:
        try:
            image = Image.open(img)
        except:
            print('Open Error! {}'.format(img))
            continue
        else:
        
            image_id = int(((img.split('/'))[-1].split('.'))[0].strip()) ##
            fullpath = os.path.join(save_dir, os.path.basename(img))
            
            detections = list()
            bounding_box_cord = list()
            label_set = list()
            score_set = list()
            speed = list()
            
            
            r_image = yolo.detect_image(image, single_image=False, output=detections, bbox_set=bounding_box_cord, label_set=label_set, score_set=score_set, speed=speed)
            
            for idx, val in enumerate(score_set):
                bbox = bounding_box_cord[idx]
                score = score_set[idx]
                category_id = label_set[idx]
                if category_id == 10:
                    category_id = 0
                detected_per_num = {
                    "image_id": image_id,
                    "bbox": bbox,
                    "score": score,
                    "category_id": category_id
                }
                submission.append(dict(detected_per_num))
            

            sub_dict["bbox"] = bounding_box_cord
            sub_dict['label'] = label_set
            sub_dict['score'] = score_set
            # print(bounding_box_cord, label_set, score_set)
            # submission.append(dict(sub_dict))
            speed_set.append(speed[0])
            if not output_txt:
                r_image.save(fullpath,"PNG")
                print('save {}'.format(fullpath))
            else:
                basename = os.path.basename(img)  # eg. 123.jpg
                txt_file = os.path.splitext(basename)[0]+'.txt'  # eg. 0001
                txt_fullpath = os.path.join(save_dir, txt_file)
                with open(txt_fullpath, 'w+') as the_file:
                    full_text = '\n'.join(detections)
                    the_file.write(full_text)

    print("Image Count: " +str(len(speed_set)))
    print("Average: " + str(1000*sum(speed_set)/ len(speed_set)) +" ms per image")
    txt_fullpath = "./answer.json"
    with open(txt_fullpath, 'w+') as json_file:
        json.dump(submission, json_file, cls=MyEncoder)
        
    # yolo.close_session()

In [5]:
class MyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        else:
            return super(MyEncoder, self).default(obj)

In [6]:
config = {
    "model_path": "logs/svhn_weights/ep006-loss29.658-val_loss24.833.h5"
}

detect_imgdir(YOLO(**config), "dataset/test_png", False)

logs/svhn_weights/ep006-loss29.658-val_loss24.833.h5 model, anchors, and classes loaded.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
logs/svhn_weights/ep006-loss29.658-val_loss24.833.h5
1.6279947069706395
save output/117.png
0.027672625961713493
save output/162.png
0.07983926602173597
save output/196.png
0.0299326159292832
save output/203.png
0.04991630790755153
save output/256.png
0.06659681105520576
save output/319.png
0.027694112970493734
save output/345.png
0.05516391701530665
save output/372.png
0.06238262203987688
save output/397.png
0.027147974004037678
save output/401.png
0.0599728130036965
save output/499.png
0.05360106402076781
save output/521.png
0.06255157198756933
save output/564.png
0.05418859503697604
save output/570.png
0.027267110999673605
save output/581.png
0.05524066498037428
save output/594.png
0.061741655925288796
save output/621.png
0.02747900108806789
save output/757.png
0.0553786150412634
save output/764.png
0.0

In [7]:
! zip 310551135_submission.zip answer.json 

updating: answer.json (deflated 66%)
